In [11]:
from stage import getStage, fullPrint
import pandas as pd
import datetime
from datetime import date, datetime
import yahoo_fin.stock_info as yf

In [12]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            #print(df.at[date,'MA30Slope'],df.at[date,'RS'])
            #!!!!!!!! or for new params
            #!!!!!!!!
            #!!!!!!!!
            #!!!!!!!!
            if  df.at[date,'MA30Slope'] > 1.0013386634337298 and df.loc[date,'RS'] > 0.07551020458494859:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1
    #print(goodSectorDf.at[date,'Sectors'])
fullPrint(goodSectorDf)

                                        Sectors
1999-01-08                                   {}
1999-01-15                                   {}
1999-01-22                                   {}
1999-01-29                                   {}
1999-02-05                                   {}
1999-02-12                                   {}
1999-02-19                                   {}
1999-02-26                                   {}
1999-03-05                                   {}
1999-03-12                                   {}
1999-03-19                                   {}
1999-03-26                                   {}
1999-04-09                                   {}
1999-04-16                                   {}
1999-04-23                                   {}
1999-04-30                                   {}
1999-05-07                                   {}
1999-05-14                                   {}
1999-05-21                                   {}
1999-05-28                              

In [13]:
# Open a file
result = open("Testdata/Realtime/result"+date.today().strftime('%Y-%m-%d')+".txt", "w")
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
buyList = []
holdList = []
sellList = []
for symbol in nasdaqList:
    df = getStage(symbol,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
    shares = 0
    sharesSPY = 0
    if df.empty:
        continue
    if df.iloc[-1].Stage=="Buy":
        print(symbol+" buy")
        result.write(symbol+" buy\n")
        buyList.append(symbol)
    elif df.iloc[-1].Stage=="Stage 2":
        print(symbol+" hold")
        result.write(symbol+" hold\n")
        holdList.append(holdList)
    elif df.iloc[-1].Stage=="Sell":
        print(symbol+" sell")
        result.write(symbol+" sell\n")
        sellList.append(sellList)
result.close()

KeyError: 'AACG'

In [ ]:
for symbol in buyList:
    df = getStage(symbol,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
    df.to_csv("realtime"+symbol+".csv")

In [ ]:
pd.read_pickle("stockData/nyseNasdaq/DXR.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'stockData/nyseNasdaq/DXR.pkl'